In [48]:
import pandas as pd

In [61]:
sch_flow = pd.read_csv("data/raw/schwartau/flow_schwartau.csv")
test_flow = sch_flow
sch_temp = pd.read_csv("data/raw/schwartau/temperature_schwartau.csv")
test_sch = sch_temp
sch_humid = pd.read_csv("data/raw/schwartau/humidity_schwartau.csv")
sch_weight = pd.read_csv("data/raw/schwartau/weight_schwartau.csv")

pls love me I need it to survive -Derek

<hr>
Cleaning notes for flow:

* Lots of zero data
* Measured every minute. Need to aggregate
* Negative numbers are bee departures
* Positive numbers are bee arrivals
* Keep arrivals and departures separate 

I separated by every 15 like the temp data and then created 2 df copies for all arrivals and all departures.  DONE-Derek

In [62]:
#original flow
print(test_flow.head())
print()

#Required changes:
#Rather than dropping all data that doesn't occur every 15 minutes, we need to sum up departures and arrivals
#that occur in each 15 minute timespan. 

#convert to every 15 min
tempList = []
sch_flow = sch_flow.dropna()
sch_flow['timestamp'] = pd.to_datetime(sch_flow['timestamp'])
    if (minute %15 != 0):
        tempList.append(ind)
sch_flow = sch_flow.drop(index = tempList)
print(sch_flow.head())
print()

#separate out arrivals and departures in separte dataframes
sch_flowArrival = sch_flow[sch_flow["flow"] > 0]
sch_flowDepart = sch_flow[sch_flow["flow"] < 0]
sch_flow = sch_flow[sch_flow["flow"] == 0]

#Take the absolute value of the departures dataframe
#This way the number of arrivals isn't a negative number
sch_flowDepart["flow"] = sch_flowDepart["flow"].abs()

#Rename the columns of the arrival and departure dataframes
sch_flowArrival.rename(columns={"flow":"arrivals"}, inplace=True)
sch_flowDepart.rename(columns={"flow":"departures"}, inplace=True)

print(sch_flowArrival.head())
print(sch_flowDepart.head())
print(sch_flow.head())

             timestamp  flow
0  2017-01-01 14:15:00     0
1  2017-01-01 14:16:00     0
2  2017-01-01 14:17:00     0
3  2017-01-01 14:18:00     0
4  2017-01-01 14:19:00     0

             timestamp  flow
0  2017-01-01 14:15:00     0
15 2017-01-01 14:30:00     0
30 2017-01-01 14:45:00     0
45 2017-01-01 15:00:00     0
60 2017-01-01 15:15:00     0

                  timestamp  arrivals
1297343 2017-01-29 16:00:00         1
1305833 2017-02-04 13:30:00         1
1323308 2017-02-16 16:45:00         1
1324598 2017-02-17 14:15:00         1
1324733 2017-02-17 16:30:00         2
               timestamp  departures
1200 2017-01-02 10:15:00           1
1590 2017-01-02 16:45:00           1
2775 2017-01-03 12:30:00           1
4155 2017-01-04 11:30:00           1
9585 2017-01-08 06:00:00           1
             timestamp  flow
0  2017-01-01 14:15:00     0
15 2017-01-01 14:30:00     0
30 2017-01-01 14:45:00     0
45 2017-01-01 15:00:00     0
60 2017-01-01 15:15:00     0


<hr>
Cleaning notes for temperature:

* May contain NaN (Dropped rows containing these.  DONE-Derek)
* Measured in degrees celsius (Will leave as C unless otherwise stated.  DONE-Derek)
* Measured every 5 minutes (coverted to every 15)

In [63]:
#Preview of temp
print(test_sch.head())
print()
#Initial number of values
print(len(test_sch.index))
#Drop NaN value rows
sch_tempFilter = test_sch.dropna()
print()
#Number of values after
print(len(sch_tempFilter.index))
print()
print(sch_tempFilter.head())
print()

#Sorting timestamp...
tempList = []
sch_temp = sch_temp.dropna()
sch_temp['timestamp'] = pd.to_datetime(sch_temp['timestamp'])
for ind,minute in sch_temp['timestamp'].dt.minute.items():
    if (minute %15 != 0):
        tempList.append(ind)
sch_temp = sch_temp.drop(index = tempList)

#sch_temp.reset_index()

print(sch_temp.head())

             timestamp  temperature
0  2017-01-01 14:10:00          NaN
1  2017-01-01 14:15:00       12.340
2  2017-01-01 14:20:00       12.270
3  2017-01-01 14:25:00       12.276
4  2017-01-01 14:30:00       12.356

253430

251398

             timestamp  temperature
1  2017-01-01 14:15:00       12.340
2  2017-01-01 14:20:00       12.270
3  2017-01-01 14:25:00       12.276
4  2017-01-01 14:30:00       12.356
5  2017-01-01 14:35:00       12.404

             timestamp  temperature
1  2017-01-01 14:15:00       12.340
4  2017-01-01 14:30:00       12.356
7  2017-01-01 14:45:00       12.492
10 2017-01-01 15:00:00       12.454
13 2017-01-01 15:15:00       12.378


/usr/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


<hr>
Cleaning notes for humidity:

* Measured in percentage
* Measured twice per day
* May need to be suplimented with other data

In [64]:
sch_humid.head()

,timestamp,humidity
0,2017-01-01 13:00:00,98.040310
1,2017-01-02 01:00:00,98.610556
2,2017-01-02 13:00:00,99.002083
3,2017-01-03 01:00:00,98.486806
4,2017-01-03 13:00:00,98.320139


<hr>
Cleaning notes for weight:

* Measured twice per day. Need some creative solution here
* Potential to correlate with flow?
* The weight for schwartau needs to be divided by 1000. It was measured in grams accidentally. DONE-Derek

In [65]:
#original data
test_weight = sch_weight
print(test_weight.head())
print()
#fixing weight
sch_weight["weight"] = sch_weight["weight"].div(1000)
print(sch_weight.head())

             timestamp        weight
0  2017-01-01 13:00:00  50736.790078
1  2017-01-02 01:00:00  50700.685000
2  2017-01-02 13:00:00  50614.907500
3  2017-01-03 01:00:00  50739.824167
4  2017-01-03 13:00:00  50799.746944

             timestamp     weight
0  2017-01-01 13:00:00  50.736790
1  2017-01-02 01:00:00  50.700685
2  2017-01-02 13:00:00  50.614908
3  2017-01-03 01:00:00  50.739824
4  2017-01-03 13:00:00  50.799747


<hr>

Finally, combine the cleaned dataset into one dataframe:

In [67]:
#Display information about the flow arrival dataset
print("Arrival dataframe:")
print("Arrival dataframe length: " + str(len(sch_flowArrival)))
print(sch_flowArrival.head())

#Display information about the flow departure dataset
print("\n\nDeparture dataframe:")
print("Departure dataframe length: " + str(len(sch_flowDepart)))
print(sch_flowDepart.head())

#Display information about the temperature dataset
print("\n\nTemperature dataframe:")
print("Temperature dataframe length: " + str(len(sch_temp)))
print(sch_temp.head())

#Merge the datasets together
sch_merged = pd.merge_asof(sch_flowDepart, sch_flowArrival, on='timestamp', by='timestamp')
sch_merged = sch_merged.sort_values(by='timestamp', ascending=True)
sch_temp = sch_temp.sort_values(by='timestamp', ascending=True)

#Display information about the merged dataset
print("\n\nMerged dataset:")
print("Merged dataset length: " + str(len(sch_merged)))
print(sch_merged.head())

sch_merged = pd.merge_asof(sch_temp, sch_merged, on='timestamp', by='timestamp')

#Display information about the merged dataset
print("\n\nMerged dataset:")
print("Merged dataset length: " + str(len(sch_merged)))
print(sch_merged.head())

#Unsure if this is valid yet. Shouldn't be, but I need to check.
print(sch_merged.loc[sch_merged['departures'] == 1])

Arrival dataframe:
Arrival dataframe length: 35845
                  timestamp  arrivals
1297343 2017-01-29 16:00:00         1
1305833 2017-02-04 13:30:00         1
1323308 2017-02-16 16:45:00         1
1324598 2017-02-17 14:15:00         1
1324733 2017-02-17 16:30:00         2


Departure dataframe:
Departure dataframe length: 36228
               timestamp  departures
1200 2017-01-02 10:15:00           1
1590 2017-01-02 16:45:00           1
2775 2017-01-03 12:30:00           1
4155 2017-01-04 11:30:00           1
9585 2017-01-08 06:00:00           1


Temperature dataframe:
Temperature dataframe length: 83800
             timestamp  temperature
1  2017-01-01 14:15:00       12.340
4  2017-01-01 14:30:00       12.356
7  2017-01-01 14:45:00       12.492
10 2017-01-01 15:00:00       12.454
13 2017-01-01 15:15:00       12.378


Merged dataset:
Merged dataset length: 36228
            timestamp  departures  arrivals
0 2017-01-02 10:15:00           1       NaN
1 2017-01-02 16:45:00         